<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/Glasgow_Range_Doppler_and_Spectrogram_BiLSTM_twoclass_augmented_duzenlendi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Burada sadece range-doppler 2D image datası ve spectrogram datası LSTM ile train edildi.
# 2D image'ın column'ları lstm'e sokuldu ve çıktıları dense layer ile classification moduna getirildi.
#Mean test accuracy is 0.999, mean test f1 score is 0.999, max test accuracy is 1.000, max test f1 score is 1.000, min test accuracy is 0.998, min test f1 score is 0.998, std of test accuracy is 0.001, std of test f1 score is 0.001
#Time elapsed through all process: 324.948, sec

In [1]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load Range-Doppler fall data
range_doppler_fall = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/range_doppler_fall.mat')
range_doppler_fall = range_doppler_fall['range_doppler_fall']
range_doppler_fall = np.transpose(range_doppler_fall, (2, 0, 1)) # label x width x height
range_doppler_fall_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/range_doppler_fall_label.mat')
range_doppler_fall_label = range_doppler_fall_label['range_doppler_fall_label']

# Load Range-Doppler walking data

range_doppler_walking = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/range_doppler_walking.mat')
range_doppler_walking = range_doppler_walking['range_doppler_walking']
range_doppler_walking = np.transpose(range_doppler_walking, (2, 0, 1))# label x width x height
range_doppler_walking_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/range_doppler_walking_label.mat')
range_doppler_walking_label = range_doppler_walking_label['range_doppler_walking_label']

# Load Spectrogram fall data
spectrogram_fall = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_fall.mat')
spectrogram_fall = spectrogram_fall['spectrogram_fall']
spectrogram_fall = np.transpose(spectrogram_fall, (2, 0, 1)) # label x width x height
spectrogram_fall_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_fall_label.mat')
spectrogram_fall_label = spectrogram_fall_label['spectrogram_fall_label']


# Load Spectrogram walking data
spectrogram_walking= scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_walking.mat')
spectrogram_walking = spectrogram_walking['spectrogram_walking']
spectrogram_walking = np.transpose(spectrogram_walking, (2, 0, 1)) # label x width x height
spectrogram_walking_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_Glasgow/alldata2/spectrogram_walking_label.mat')
spectrogram_walking_label = spectrogram_walking_label['spectrogram_walking_label']


In [4]:
# Concat range-doppler data
range_doppler_concat = np.concatenate((range_doppler_fall,range_doppler_walking),axis=0) # fast =fall, slow=walking
range_doppler_concat = range_doppler_concat[:,:,:,np.newaxis]
range_doppler_concat_label = np.zeros((range_doppler_concat.shape[0],1))
range_doppler_concat_label[:range_doppler_fall.shape[0],:] = 1
# Shuffle concat range doppler
shuffle_indx = random.sample(range(0, range_doppler_concat.shape[0]), range_doppler_concat.shape[0]) # split validation data
range_doppler_concat_shuffle = range_doppler_concat[shuffle_indx,:,:,:]
range_doppler_concat_label_shuffle = range_doppler_concat_label[shuffle_indx,:]
# Concat spectrogram data
spectrogram_concat = np.concatenate((spectrogram_fall,spectrogram_walking),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]
spectrogram_concat_label = np.zeros((spectrogram_concat.shape[0],1))
spectrogram_concat_label[:spectrogram_fall.shape[0],:] = 1
# Shuffle concat spectrogram
spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]

In [12]:

def mixup_augmentation(images,training_data, labels, repeat_of_mixup, alpha):
    batch_size = images.shape[0]
    concat_images = np.zeros((batch_size*(repeat_of_mixup+1),images.shape[1],images.shape[2],images.shape[3]))
    concat_images_range_doppler = np.zeros((batch_size*(repeat_of_mixup+1),training_data.shape[1],\
                                            training_data.shape[2],training_data.shape[3]))
    concat_label = np.zeros((batch_size*(repeat_of_mixup+1),labels.shape[1]))
    for ii in range(repeat_of_mixup):
      # shuffle train dataset
      shuffle_indx_1 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_1 = images[shuffle_indx_1,:,:,:]
      training_data_shuffled_1 = training_data[shuffle_indx_1,:,:,:]
      labels_shuffled_1 = labels[shuffle_indx_1,:]

      shuffle_indx_2 = random.sample(range(0, images.shape[0]), images.shape[0]) # split validation data
      images_shuffled_2 = images[shuffle_indx_2,:,:,:]
      training_data_shuffled_2 = training_data[shuffle_indx_2,:,:,:]
      labels_shuffled_2 = labels[shuffle_indx_2,:]

      # Sample lambda from beta distribution
      ll=np.random.beta(alpha,alpha, (batch_size,1,1,1))
      x_l = np.reshape(ll, (batch_size,1,1,1))
      y_l = np.reshape(ll, (batch_size,1))

      # Perform mixup on both images and labels by combining a pair of images/labels
      images_mixup = images_shuffled_1 * x_l + images_shuffled_2 * (1 - x_l)
      images_mixup_range_doppler = training_data_shuffled_1 * x_l + training_data_shuffled_2 * (1 - x_l)
      labels_mixup = labels_shuffled_1 * y_l + labels_shuffled_2 * (1 - y_l)
      concat_images[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup
      concat_images_range_doppler[ii*batch_size:(ii+1)*batch_size,:,:,:] = images_mixup_range_doppler
      concat_label[ii*batch_size:(ii+1)*batch_size,:] = labels_mixup

    concat_images[repeat_of_mixup*batch_size:,:,:,:] = images
    concat_images_range_doppler[repeat_of_mixup*batch_size:,:,:,:] = training_data
    concat_label[repeat_of_mixup*batch_size:,:] = labels
    return (concat_images,concat_images_range_doppler, concat_label)





def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,label_shuffle_train):

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx

  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  range_doppler_validation_data = range_doppler_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]

  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  range_doppler_training_data = range_doppler_concat_shuffle_train[randomlist_for_train_indx,:,:,:]


  #get labels
  validation_labels = label_shuffle_train[randomlist_for_validation_indx,:]
  training_labels = label_shuffle_train[randomlist_for_train_indx,:]


  # get CLASS1 and CLASS2 indexes of training dat

  class1_indexes = np.where(training_labels == 0)[0]
  class2_indexes = np.where(training_labels == 1)[0]
  class3_indexes = np.where(training_labels == 2)[0]


  class1_spectrograms_train = spectrogram_training_data[class1_indexes,:,:,:]
  class2_spectrograms_train = spectrogram_training_data[class2_indexes,:,:,:]
  class3_spectrograms_train = spectrogram_training_data[class3_indexes,:,:,:]


  class1_rangedoppler_train = range_doppler_training_data[class1_indexes,:,:,:]
  class2_rangedoppler_train = range_doppler_training_data[class2_indexes,:,:,:]
  class3_rangedoppler_train = range_doppler_training_data[class3_indexes,:,:,:]


  size_of_samples_class1= class1_spectrograms_train.shape[0]
  size_of_samples_class2 = class2_spectrograms_train.shape[0]
  size_of_samples_class3 = class3_spectrograms_train.shape[0]


  class1_label = np.zeros((size_of_samples_class1,1))
  class2_label = np.ones((size_of_samples_class2,1))
  class3_label = 2*np.ones((size_of_samples_class3,1))


  # unify data
  spectrogram_training_data = np.concatenate((class3_spectrograms_train,class2_spectrograms_train,class1_spectrograms_train),axis=0)
  range_doppler_training_data = np.concatenate((class3_rangedoppler_train,class2_rangedoppler_train,class1_rangedoppler_train),axis=0)


  training_labels = np.concatenate((class3_label,class2_label,class1_label),axis=0)

  (spectrogram_augmented_image,range_doppler_augmented_image,training_labels_final)=mixup_augmentation(spectrogram_training_data,range_doppler_training_data, training_labels, repeat_of_mixup, alpha)





  return (spectrogram_augmented_image,range_doppler_augmented_image,training_labels_final,\
     spectrogram_validation_data,range_doppler_validation_data, validation_labels)

In [9]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)

In [10]:
normalize_inputs_enable = 1
range_doppler_concat_shuffle = normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
range_doppler_concat_shuffle = np.float32(range_doppler_concat_shuffle)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [11]:

range_doppler_concat_shuffle = np.transpose(range_doppler_concat_shuffle, axes = (0,2,1,3))
spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))

In [14]:
t = time.time()
# ---------- Parameters ----------------
alpha=0.2
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 10
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 256
dense_unit_of_spectrogram_function = 8
decoder_dense_unit = 256
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del range_doppler_concat_shuffle_test
    del spectrogram_concat_shuffle_test
    del range_doppler_augmented_image
    del range_doppler_concat_shuffle_train
    del spectrogram_concat_shuffle_train
    del spectrogram_augmented_image

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(range_doppler_concat_shuffle,range_doppler_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    range_doppler_concat_shuffle_test = range_doppler_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    range_doppler_concat_label_shuffle_test = range_doppler_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    range_doppler_concat_shuffle_train = range_doppler_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (spectrogram_augmented_image,range_doppler_augmented_image,spectrogram_concat_label_shuffle_concat,\
     validation_spectrogram,validation_range_doppler, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,range_doppler_concat_shuffle_train,\
                                         spectrogram_concat_label_shuffle_train)

    # ---------------- Neural Network Architecture ----------------



    def lstm_encoder_network_1(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_range_doppler_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def lstm_encoder_network_2(input_shape):
        input = Input(shape=input_shape)
        x = Bidirectional(LSTM(unit_number_of_lstm, return_sequences=True, dropout = 0.5))(input)
        x = Flatten()(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(dense_unit_of_spectrogram_function)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(dropout_prob_dense)(x)
        return Model(input, x)

    def decoder_for_concat(input_shape):
      input = Input(shape=input_shape)
      x = Dense(decoder_dense_unit)(input)
      x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(0.3)(x)
      x = Dense(dense_size)(x)
      # x = BatchNormalization()(x)
      x = LeakyReLU()(x)
      x = Dropout(dropout_prob_dense)(x)
      x = Dense(1, activation="sigmoid")(x)
      return Model(input, x)

    input_shape = range_doppler_concat_shuffle.shape[1:3]
    base_network_lstm = lstm_encoder_network_1(input_shape)
    range_doppler_input  = Input(shape=input_shape)
    processed_range_doppler  = base_network_lstm(range_doppler_input)

    input_shape = spectrogram_concat_shuffle_train.shape[1:3]
    base_network_lstm_2 = lstm_encoder_network_2(input_shape)
    spectrogram_input  = Input(shape=input_shape)
    processed_spectrogram  = base_network_lstm_2(spectrogram_input)

    concat_layer = Concatenate()([processed_range_doppler, processed_spectrogram])

    base_decoder_network = decoder_for_concat((concat_layer.shape[1],))
    out = base_decoder_network(concat_layer)

    model = Model(inputs=[range_doppler_input, spectrogram_input], outputs=[out])
    if repeat_run_number == 0:
      print(base_network_lstm.summary())
      print(base_network_lstm_2.summary())
      print(base_decoder_network.summary())
    # ---------------- Compile and Fit ----------------
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')
    history = model.fit((range_doppler_augmented_image, spectrogram_augmented_image),(spectrogram_concat_label_shuffle_concat),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((validation_range_doppler, validation_spectrogram) , (spectrogram_validation_labels)))
    test_loss, test_accuracy  = model.evaluate([range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test],\
                                               [range_doppler_concat_label_shuffle_test],
                  batch_size=batch_size)
    gc.collect()
    # ---------------- Get Test Results ----------------
    y_test_predicted = model.predict((range_doppler_concat_shuffle_test, spectrogram_concat_shuffle_test), batch_size=batch_size)
    # ----- Binarize y_test_predicted values -----
    y_test_predicted_binary = np.zeros(y_test_predicted.size)
    for ii in range(y_test_predicted.size):
      if y_test_predicted[ii] < 0.5:
        y_test_predicted_binary[ii] = 0
      else:
        y_test_predicted_binary[ii] = 1

    test_precision, test_recall, test_f1_score, support = precision_recall_fscore_support(range_doppler_concat_label_shuffle_test, y_test_predicted_binary, average='macro')

    test_accuracy_per_fold.append(test_accuracy)
    f1_score_per_fold.append(test_f1_score)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  f1_score_per_run.append(sum(f1_score_per_fold)/num_folds)
  print(test_accuracy_per_run)
  print(f1_score_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100, 100)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 32)        │        14,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 3200)           │        12,800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       819,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 848,256 (3.24 MB)

 Trainable params: 841,344 (3.21 MB)

 Non-trainable params: 6,912 (27.00 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 264)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        67,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 85,377 (333.50 KB)

 Trainable params: 84,865 (331.50 KB)

 Non-trainable params: 512 (2.00 KB)

None
Epoch 1/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.4072 - loss: 0.6852 - val_accuracy: 0.9861 - val_loss: 0.1552
Epoch 2/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5511 - loss: 0.3712 - val_accuracy: 1.0000 - val_loss: 0.0564
Epoch 3/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5701 - loss: 0.2449 - val_accuracy: 0.9861 - val_loss: 0.0342
Epoch 4/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.5829 - loss: 0.2205 - val_accuracy: 0.9861 - val_loss: 0.0278
Epoch 5/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5601 - loss: 0.2196 - val_accuracy: 0.9861 - val_loss: 0.0238
Epoch 6/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5867 - loss: 0.1979 - val_accuracy: 0.9861 - val_loss: 0.0235
Epoch 7/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.5936 - loss: 0.1866 - val_accuracy: 1.0000 - val_loss: 0.0153
Epoch 8/10
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.5978 - loss: 0.1732 - val_accuracy: 1.000

In [15]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

Mean test accuracy is 0.999, mean test f1 score is 0.999, max test accuracy is 1.000, max test f1 score is 1.000, min test accuracy is 0.998, min test f1 score is 0.998, std of test accuracy is 0.001, std of test f1 score is 0.001
Time elapsed through all process: 324.948, sec
